In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
plt.style.use('fivethirtyeight')

In [2]:
# read in sp500 ticker history dataframe to build inclusion matrix
sp500_history_df = pd.read_csv('sp500_history.csv')
sp500_history_df.head()

,Unnamed: 0,cik,date,name,value,variable
0,183,72741.0,1957-01-01,Eversource Energy,ES,added_ticker
1,228,874766.0,1957-01-01,Hartford Financial Svc.Gp.,HIG,added_ticker
2,435,1113169.0,1957-01-01,T. Rowe Price Group,TROW,added_ticker
3,349,1111711.0,1957-01-01,NiSource Inc.,NI,added_ticker
4,185,1109357.0,1957-01-01,Exelon Corp.,EXC,added_ticker


In [16]:
tickers = np.unique(sp500_history_df['value'])
print("Number of unique tickers:", len(tickers))

Number of unique tickers: 713


In [27]:
status = np.unique(sp500_history_df['variable'])
status

array(['added_ticker', 'removed_ticker'], dtype=object)

In [5]:
# check the type of date column
type(sp500_history_df['date'][0])

str

In [7]:
# convert the date column into datetime format
sp500_history_df.loc[:,'date'] = pd.to_datetime(sp500_history_df['date'])
type(sp500_history_df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
date_index = pd.date_range(start=sp500_history_df['date'][0], end=pd.to_datetime('today'), freq='B')
date_index

DatetimeIndex(['1957-01-01', '1957-01-02', '1957-01-03', '1957-01-04',
               '1957-01-07', '1957-01-08', '1957-01-09', '1957-01-10',
               '1957-01-11', '1957-01-14',
               ...
               '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23',
               '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01',
               '2021-03-02', '2021-03-03'],
              dtype='datetime64[ns]', length=16742, freq='B')

In [26]:
test_df = sp500_history_df[(sp500_history_df['date'] == pd.to_datetime('1957-01-01'))]
dict(zip(test_df['value'], test_df['variable']))

{'ES': 'added_ticker',
 'HIG': 'added_ticker',
 'TROW': 'added_ticker',
 'NI': 'added_ticker',
 'EXC': 'added_ticker',
 'SO': 'added_ticker',
 'MET': 'added_ticker',
 'CVX': 'added_ticker',
 'STT': 'added_ticker',
 'TXN': 'added_ticker',
 'TMO': 'added_ticker',
 'TSN': 'added_ticker',
 'UNP': 'added_ticker',
 'UTX': 'added_ticker',
 'EBAY': 'added_ticker',
 'MCO': 'added_ticker',
 'PG': 'added_ticker',
 'WY': 'added_ticker',
 'PPG': 'added_ticker',
 'PEP': 'added_ticker',
 'IP': 'added_ticker',
 'K': 'added_ticker',
 'KMB': 'added_ticker',
 'KR': 'added_ticker',
 'LEG': 'added_ticker',
 'L': 'added_ticker',
 'MKC': 'added_ticker',
 'SPGI': 'added_ticker',
 'CVS': 'added_ticker',
 'COP': 'added_ticker',
 'ETR': 'added_ticker',
 'MMM': 'added_ticker',
 'MSI': 'added_ticker',
 'XEL': 'added_ticker',
 'NTRS': 'added_ticker',
 'PFE': 'added_ticker',
 'WHR': 'added_ticker',
 'XRX': 'added_ticker',
 'MAR': 'added_ticker',
 'MO': 'added_ticker',
 'PPL': 'added_ticker',
 'PNW': 'added_ticker',


In [30]:
sp500_history_df[(sp500_history_df['value'] == 'AAPL')]

,Unnamed: 0,cik,date,name,value,variable
149,49,320193.0,1982-11-30,Apple Inc.,AAPL,added_ticker


In [41]:
inclusion_matrix = [[len(sp500_history_df[(sp500_history_df['date'] == pd.to_datetime('1957-01-01')) & (sp500_history_df['value']==ticker)]) for ticker in tickers]]


In [42]:
for date in date_index[1:]:
    df_at_date = sp500_history_df[sp500_history_df['date'] == date]
    ticker_status = dict(zip(df_at_date['value'], df_at_date['variable']))
    inclusion_matrix.append([0 for _ in tickers])
    for i in range(len(tickers)):
        if tickers[i] in ticker_status:
            if ticker_status[tickers[i]] == 'added_ticker':
                inclusion_matrix[-1][i] = 1
            else:
                inclusion_matrix[-1][i] = 0
        else:
            inclusion_matrix[-1][i] = inclusion_matrix[-2][i]


In [43]:
inclusion_df = pd.DataFrame(index = date_index, columns=tickers, data=inclusion_matrix)
inclusion_df

,A,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABT,ACAS,...,XOM,XRAY,XRX,XTO,XYL,YHOO,YUM,ZBH,ZION,ZTS
1957-01-01,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1957-01-02,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1957-01-03,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1957-01-04,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1957-01-07,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-25,1,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,0,1,1,1,1
2021-02-26,1,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,0,1,1,1,1
2021-03-01,1,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,0,1,1,1,1
2021-03-02,1,1,1,1,1,1,0,1,1,0,...,1,1,1,0,1,0,1,1,1,1


In [44]:
inclusion_df.to_csv('sp500_inclusion_matrix_20200303.csv')